In [ ]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim

In [ ]:
# Additional dependencies

# For parsing the data from the API
import json
# For downloading data from API
import requests as req
# For generating random numbers
import math
# For visualization
import plotly.express as px
# For Google BigQuery authentication
from google.oauth2 import service_account

In [ ]:
%%capture

credentials = service_account.Credentials.from_service_account_file(
    './credentials.json',
)

QUERY = """
SELECT * FROM blockchain-etl.ethereum_balancer.V2_Vault_event_PoolBalanceChanged
WHERE poolId = "0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f8000200000000000000000019"
ORDER BY block_timestamp DESC
"""

# Send the SQL query to the ethereum-etl dataset
# on Google BigQuery.
# Requires the pandas-gbq library
supply_data = pd.read_gbq(QUERY, project_id="blockchain-319713", credentials=credentials)

# Print the last 5 rows
supply_data.tail(5)

In [ ]:
supply_data

In [ ]:
%%capture

credentials = service_account.Credentials.from_service_account_file(
    './credentials.json',
)

QUERY = """
SELECT * FROM blockchain-etl.ethereum_balancer.V2_Vault_event_Swap
WHERE poolId = "0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f8000200000000000000000019"
ORDER BY block_timestamp DESC
"""

# Send the SQL query to the ethereum-etl dataset
# on Google BigQuery.
# Requires the pandas-gbq library
swap_data = pd.read_gbq(QUERY, project_id="blockchain-319713", credentials=credentials)

# Print the last 5 rows
swap_data.tail(5)

In [ ]:
tokens = {
    "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48": {
        "ticker": "UDSC",
        "decimal": 6
    },
    "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2": {
        "ticker": "WETH",
        "decimal": 18
    }
}

In [ ]:
swap_data

In [ ]:
swap_data[]

### Initialize pool

In [ ]:
from decimal import *
from BalancerV2cad.WeightedPool import WeightedPool

In [ ]:
wp = WeightedPool()
wp._swap_fee = Decimal(0.003)
wp.join_pool({'a':100,'b':100},{'a':0.5,'b':0.5})

print(wp.swap('b', 'a', 1, given_in=False))
print(wp._balances['a'], wp._balances['b'], wp.factory_fees)

In [ ]:
from BalancerV2cad.WeightedMath import WeightedMath

In [ ]:
# calc_in_given_out(balance_in: Decimal,
#                           weight_in: Decimal,
#                           balance_out: Decimal,
#                           weight_out: Decimal,
#                           amount_out: Decimal):

In [ ]:
def changePoolWeights(wp, new_weights):
    new_pool = WeightedPool()
    balances = wp._balances
    new_pool.join_pool(balances,new_weights)
    new_pool.factory_fees = wp.factory_fees
    return new_pool

updated_wp = changePoolWeights(wp, {'a': 0.2, 'b': 0.8})

In [ ]:
updated_wp._weights

In [ ]:
updated_wp.factory_fees

In [ ]:
from scipy.optimize import fsolve
import numpy as np
def func(x):
    return [x[0] * np.cos(x[1]) - 4,
            x[1] * x[0] - x[1] - 5]
root = fsolve(func, [1, 1])
print(root)

In [ ]:
from scipy.optimize import fsolve
import numpy as np

a_bal = 100
a_weight = 0.5
b_bal = 100
b_weight = 0.5
new_price = 4.34739457389

def func(x):
    return [x[0] + x[1] - 1,
            a_bal*x[1] - new_price*b_bal*x[0]]
root = fsolve(func, [0.5, 0.5])
print(root)

In [ ]:
# Stop when difference between weight-adjusted price is within 0.05% of new_price (< 0.0005x)
def find_optimal_weights(a_bal, a_weight, b_bal, b_weight, new_price):
    def func(x):
        return [x[0] + x[1] - 1,
            b_bal*x[0] - new_price*a_bal*x[1]]
    root = fsolve(func, [0.5, 0.5])
    return root



find_optimal_weights(100, 0.5, 100, 0.5, 4)

## Chainlink Price Data

In [ ]:
from web3 import Web3

# Change this to use your own Infura ID
web3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/1483415a8185479793708205947c7080'))
# AggregatorV3Interface ABI
abi = '[{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]'
# Price Feed address
addr = '0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419'

# Set up contract instance
contract = web3.eth.contract(address=addr, abi=abi)

#  Valid roundId must be known. They are NOT incremental.
# invalidRoundId = 18446744073709562300
validRoundId = 92233720368547765431

historicalData = contract.functions.getRoundData(validRoundId).call()
print(historicalData)

In [ ]:
web3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/1483415a8185479793708205947c7080'))
abi = '[{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]'
addr = '0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419'
contract = web3.eth.contract(address=addr, abi=abi)
latestData = contract.functions.latestRoundData().call()
print(latestData)

In [ ]:
# For parsing the data from the API
import json
# For downloading data from API
import requests as req

# You can explore the subgraph at https://thegraph.com/explorer/subgraph/balancer-labs/balancer
API_URI = 'https://api.thegraph.com/subgraphs/name/tomafrench/chainlink'

# Query for retrieving the history of swaps on a BAL <> UNI 50-50 pool
GRAPH_QUERY = '''
{
    prices(id: "0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419", orderBy: blockNumber, where:{blockNumber_gt: "9411668"}) {
        id
        blockNumber
        price
    }
}
'''

# Retrieve data from query
JSON = {'query': GRAPH_QUERY}
r = req.post(API_URI, json=JSON)
graph_data = json.loads(r.content)['data']

print("Print first 500 characters of the response")
print(r.text[:500])

In [ ]:
n = [int(i['blockNumber']) for i in graph_data['prices']]

In [ ]:
print(sorted(n)[0], sorted(n)[-1])

In [ ]:
9152406 9411668

In [ ]:
# For parsing the data from the API
import json
# For downloading data from API
import requests as req
# For reducing calls/second to API
import time

def getPricesSinceBlockNumber(blockNumber):
    API_URI = 'https://api.thegraph.com/subgraphs/name/tomafrench/chainlink'

    GRAPH_QUERY = """
{
  prices(where:{ assetPair: "USDC/ETH", blockNumber_gte: "%d"}, orderBy: blockNumber) {
    id
    blockNumber
    price
    timestamp
    priceDeviation
    timeSincePreviousPrice
  }
}
    """ % (blockNumber)
    # Retrieve data from query
    JSON = {'query': GRAPH_QUERY}
    r = req.post(API_URI, json=JSON)
    graph_data = json.loads(r.content)['data']['prices']
    return graph_data

In [ ]:
prices = []
blockNumbers = []
timestamps = []
earliestBlockNumber = 12502801

blockNumber = earliestBlockNumber

i = 0
while True:
    start = blockNumber
    graph_data = getPricesSinceBlockNumber(blockNumber)
    
    if len(graph_data) == 0:
        break
    else:
        prices += graph_data
        blockNumbers += sorted([int(i['blockNumber']) for i in graph_data])
        timestamps += sorted([int(i['timestamp']) for i in graph_data])
        blockNumber = blockNumbers[-1]+1
        print("Fetched {}-{}".format(start, blockNumber))
        time.sleep(0.5)
    i += 1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot([int(i['timestamp']) for i in prices], [1/(int(i['price'])/1e18) for i in prices])

In [ ]:
len(prices)

In [ ]:
(max(blockNumbers) - min(blockNumbers))

In [ ]:
[i['price'] for i in prices]

In [ ]:
from scipy.optimize import fsolve
import numpy as np

a_bal = 100
a_weight = 0.5
b_bal = 100
b_weight = 0.5
new_price = 4.34739457389

def func(x):
    return [x[0] + x[1] - 1,
            a_bal*x[1] - new_price*b_bal*x[0]]
root = fsolve(func, [0.5, 0.5])
print(root)

In [ ]:
from decimal import *
from BalancerV2cad.WeightedPool import WeightedPool
from BalancerV2cad.WeightedMath import WeightedMath

from BalancerV2cad.util import *
from BalancerV2cad.BalancerConstants import *

wp = WeightedPool()
wp._swap_fee = Decimal(0.003)
wp.join_pool({'a':100,'b':100*150},{'a':0.5,'b':0.5})

print(wp._balances['a'], wp._balances['b'], wp.factory_fees)

In [ ]:
# WeightedMath.calc_out_given_in(balance_in: Decimal, 
#                           weight_in: Decimal,
#                           balance_out: Decimal,
#                           weight_out: Decimal,
#                           amount_in: Decimal) 

WeightedMath.calc_out_given_in(wp._balances['a'],wp._weights['a'],wp._balances['b'],wp._weights['b'],1)

In [ ]:
def func(x):
    cex_price=145.0
    return [
            divDown(WeightedMath.calc_out_given_in(wp._balances['a'],wp._weights['a'],wp._balances['b'],wp._weights['b'],Decimal(x[0])), Decimal(x[0]))-Decimal(cex_price)]
fsolve(func, [1.0])

In [ ]:
num =wp._balances['b'] / wp._weights['b']
denom = wp._balances['a'] / wp._weights['a']
num/denom

In [ ]:
wp.swap('a', 'b', 3.44827586, given_in=True)

In [ ]:
498.5541925515715067475568890/3.44827586

In [ ]:
num =wp._balances['b'] / wp._weights['b']
denom = wp._balances['a'] / wp._weights['a']
num/denom

## Trying to maximise arbitrager return.

Need to approach the problem of figuring out $A_o$ and $A_i$ such that any additional $A_i+\delta$ results in a lower rate than the $cex$

In [ ]:
from decimal import *
from BalancerV2cad.WeightedPool import WeightedPool
from BalancerV2cad.WeightedMath import WeightedMath

from BalancerV2cad.util import *
from BalancerV2cad.BalancerConstants import *

wp = WeightedPool()
wp._swap_fee = Decimal(0.003)
wp.join_pool({'a':100,'b':100*150},{'a':0.5,'b':0.5})

print(wp._balances['a'], wp._balances['b'], wp.factory_fees)

In [ ]:
num =wp._balances['b'] / wp._weights['b']
denom = wp._balances['a'] / wp._weights['a']
num/denom

In [ ]:
cex_price=155.0

In [ ]:
def func(x):
    return [
            divDown(
                divDown(
                    wp._balances['b']+mulDown( #Num
                        Decimal(x[0]), 1-wp._swap_fee), #Num
                    wp._weights['b'] #Denom
                ), 
                divDown( #Denom
                    wp._balances['a']-WeightedMath.calc_out_given_in(
                        wp._balances['b'],
                        wp._weights['b'],
                        wp._balances['a'],
                        wp._weights['a'],
                        Decimal(x[0]) #Num
                    ),
                    wp._weights['a']))-Decimal(cex_price) #Denom
    ]
amount = Decimal(fsolve(func, [1.0])[0])
print(amount)

In [ ]:
WeightedMath.calc_out_given_in(
                        wp._balances['b'],
                        wp._weights['b'],
                        wp._balances['a'],
                        wp._weights['a'],
                        Decimal(1000))

In [ ]:
def func(x):
    return [
            divDown(divDown(wp._balances['b']+Decimal(x[0]), wp._weights['b']), divDown(wp._balances['a']-WeightedMath.calc_out_given_in(wp._balances['b'],wp._weights['b'],wp._balances['a'],wp._weights['a'],Decimal(x[0])), wp._weights['a']))-Decimal(cex_price)
    ]
fsolve(func, [1.0])

In [ ]:
wp.swap('b', 'a', amount, given_in=True)

In [ ]:
num =(wp._balances['b']-wp.factory_fees['b']) / wp._weights['b']
denom = wp._balances['a'] / wp._weights['a']
num/denom

In [ ]:
wp.factory_fees

In [ ]:
def calcArbOp(wp, cex_price, cex_trade_fee, cex_trade_slippage):
    
    num =wp._balances['b'] / wp._weights['b']
    denom = wp._balances['a'] / wp._weights['a']
    current_pool_price = num/denom
    
    # Buying ETH from the pool. Selling on cex.
    if current_pool_price < cex_price:
        effective_cex_price = (1-cex_trade_fee-cex_trade_slippage)*cex_price
        trade_vol_min = Decimal(0)
        trade_vol_max = wp._balances['b']
        test_amount_in = (trade_vol_min+trade_vol_max)/2
        
        i = 0
        while i < 10:
            amount_out = WeightedMath.calc_out_given_in(wp._balances['b'],wp._weights['b'],wp._balances['a'],wp._weights['a'],test_amount_in)
            
            num =(wp._balances['b']+test_amount_in) / wp._weights['b']
            denom = (wp._balances['a']-amount_out) / wp._weights['a']
            new_pool_price = num/denom
            print(test_amount_in, new_pool_price)
            i += 1

In [ ]:
from decimal import *
from BalancerV2cad.WeightedPool import WeightedPool
from BalancerV2cad.WeightedMath import WeightedMath

from BalancerV2cad.util import *
from BalancerV2cad.BalancerConstants import *

wp = WeightedPool()
wp._swap_fee = Decimal(0.003)
wp.join_pool({'a':100,'b':100*150},{'a':0.5,'b':0.5})

print(wp._balances['a'], wp._balances['b'], wp.factory_fees)


In [ ]:
calcArbOp(wp, 155, 0.001, 0.001)

## Attempt #2

In [ ]:
def calcArbOp(wp, cex_price, cex_trade_fee, cex_trade_slippage):
    num =wp._balances['b'] / wp._weights['b']
    denom = wp._balances['a'] / wp._weights['a']
    current_pool_price = num/denom
    
    # Buying ETH from the pool. Selling on cex.
    if current_pool_price < cex_price:
        effective_cex_price = (1-cex_trade_fee-cex_trade_slippage)*cex_price
        
        def func(x):
            return [
                    divDown(
                        divDown(
                            wp._balances['b']+mulDown( #Num
                                Decimal(x[0]), 1+wp._swap_fee), #Num
                            wp._weights['b'] #Denom
                        ), 
                        divDown( #Denom
                            wp._balances['a']-WeightedMath.calc_out_given_in(
                                wp._balances['b'],
                                wp._weights['b'],
                                wp._balances['a'],
                                wp._weights['a'],
                                Decimal(x[0]) #Num
                            ),
                            wp._weights['a']))-Decimal(effective_cex_price) #Denom
            ]
        amountIn = Decimal(fsolve(func, [1.0])[0])
        return {
            'assetIn': 'b',
            'assetOut': 'a',
            'amountIn': amountIn
        }
    
    # Buying ETH on cex. Selling to the pool.
    elif current_pool_price > cex_price:
        effective_cex_price = (1+cex_trade_fee+cex_trade_slippage)*cex_price
        
        def func(x):
            return [
                    divDown(
                        divDown(
                            wp._balances['b']-WeightedMath.calc_out_given_in(
                                wp._balances['a'],
                                wp._weights['a'],
                                wp._balances['b'],
                                wp._weights['b'],
                                Decimal(x[0])
                            ), #Num
                            wp._weights['b'] #Denom
                        ), 
                        divDown( #Denom
                            wp._balances['a']+mulDown(
                                Decimal(x[0]), 1+wp._swap_fee), #Num
                            wp._weights['a']))-Decimal(effective_cex_price) #Denom
            ]
        amountIn = Decimal(fsolve(func, [1.0])[0])
        return {
            'assetIn': 'a',
            'assetOut': 'b',
            'amountIn': amountIn
        }
        
        


In [ ]:
from decimal import *
from BalancerV2cad.WeightedPool import WeightedPool
from BalancerV2cad.WeightedMath import WeightedMath

from BalancerV2cad.util import *
from BalancerV2cad.BalancerConstants import *

wp = WeightedPool()
wp._swap_fee = Decimal(0.003)
wp.join_pool({'a':100,'b':100*150},{'a':0.5,'b':0.5})

print(wp._balances['a'], wp._balances['b'], wp.factory_fees)


In [ ]:
num =wp._balances['b'] / wp._weights['b']
denom = wp._balances['a'] / wp._weights['a']
current_pool_price = num/denom
print(current_pool_price)
print(wp._balances)
print(calcArbOp(wp, 150, 0.001, 0.001))

In [ ]:
trade = calcArbOp(wp, 155, 0.001, 0.001)
print(trade)
wp.swap(trade['assetIn'], trade['assetOut'], trade['amountIn'], given_in=True)

num =wp._balances['b'] / wp._weights['b']
denom = wp._balances['a'] / wp._weights['a']
current_pool_price = num/denom
print(current_pool_price)

In [ ]:
wp._balances

In [ ]:
wp.factory_fees